In [ ]:
def get_education (level):
    level = ' '.join(level.split(' ')[:3])
    if 'Высшее' in level:
        return 'высшее'
    elif 'Неоконченное высшее' in level:
        return 'неоконченное высшее'
    elif 'Среднее специальное' in level:
        return 'среднее специальное'
    elif 'Среднее образование' in level:
        return 'среднее'
hh_data['Образование'] = hh_data['Образование и ВУЗ'].apply(get_education)
hh_data = hh_data.drop('Образование и ВУЗ', axis=1)

#hh_data['Образование'].apply(lambda x: ' '.join(x.split(' ')[:2]))

In [ ]:
#создаем новый признак 'Город'
#через лямбду вырезаем первое слово и сохраняем в новый признак 
hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(lambda x: ' '.join(x.split(' ')[:1]))
#прописываем функцию которая преобразует новый признак "Город" в 4 категории 
def get_category(arg):
    million_cities = [
        'Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', \
            'Самара', 'Ростов-на-Дону','Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' 
        ]
    if arg in million_cities:
        return 'город-миллионник'
    elif arg == 'Москва':
        return 'Москва'
    elif arg == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    else:
        return "другие"  
#применяем данную функцию     
hh_data['Город'] = hh_data['Город'].apply(get_category)
#проверяем, должно быть 4 уникальных значений
print(hh_data['Город'].nunique()) #4

#создаем новый признак 'Готовность к переезду'
hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(lambda x: x.split(',')[1])
#прописываем функцию которая преобразует новый признак "Готовность к переезду"
def get_ready_to_moov(arg):
    if ('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    elif 'хочу' in arg:
        return True
    else:
        return True
#применяем данную функцию
hh_data['Готовность к переезду'] = hh_data['Готовность к переезду'].apply(get_ready_to_moov)
print(hh_data['Готовность к переезду'].nunique())

#создаем новый признак 'Готовность к командировкам'
hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(lambda x: ' '.join(x.split(',')[2:]))
def get_ready_for_trips(arg):
    if ('командировка' in arg):
        if ('не готов к командировкам' in arg) or ('не готова к командировкам' in arg):
            return False
        else:

            return True
    else:
        return False
#применяем данную функцию
hh_data['Готовность к командировкам'] = hh_data['Готовность к командировкам'].apply(get_ready_for_trips)
print(hh_data['Готовность к командировкам'].nunique())


In [ ]:
def get_city(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород',
                      'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону',
                      'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград'
                      ]
    city = arg.split(' , ')[0]
    if (city == 'Москва') or (city == 'Санкт-Петербург'):
        return city
    elif city in million_cities:
        return 'город миллионник'
    else:
        return 'другие'


def get_ready_to_move(arg):
    if ('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    elif 'хочу' in arg:
        return True
    else:
        return True


def get_ready_for_bisiness_trips(arg):
    if ('командировка' in arg):
        if ('не готов к командировкам' in arg) or ('не готова к командировкам' in arg):
            return False
        else:

            return True
    else:
        return False


hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(get_city)
hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(get_ready_to_move)
hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(get_ready_for_bisiness_trips)
hh_data = hh_data.drop('Город, переезд, командировки', axis=1)
print(round(hh_data['Город'].value_counts(normalize=True)['Санкт-Петербург'] * 100))
print(round(hh_data[
                hh_data['Готовность к переезду'] & hh_data['Готовность к командировкам']
                ].shape[0] / hh_data.shape[0] * 100))

In [ ]:
rates = pd.read_csv('data/ExchangeRates.csv')
rates = rates.drop(['per','time', 'vol'], axis=1)

rates['date'] = pd.to_datetime(rates['date'], dayfirst=True).dt.date
hh_data['Обновление резюме'] = pd.to_datetime(hh_data['Обновление резюме'], dayfirst=True).dt.date

# создаём столбцы с названием валюты и желаемой зп
hh_data['валюта'] = hh_data['ЗП'].apply(lambda x: str(x.split(' ')[1]))
hh_data['желаемая ЗП'] = hh_data['ЗП'].apply(lambda y: y.split(' ')[0])
#убираем точки в названии валюты
hh_data['желаемая ЗП'].replace('.', '')
#Создаем словарь для валюты в ISO
currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': 'BYN',
        'EUR': 'EUR', 'KGS': 'KGS',
        'сум': 'UZS', 'AZN': 'AZN', 'руб': 'RUB'
    }
#Применяем словарь к таблице
hh_data['валюта'] = hh_data['валюта'].map(currency_dict)
#hh_data.dropna(inplace=True)
# соединим таблицы hh_data и rates
hh_data = hh_data.merge(right = rates,
                         how = 'left',
                         left_on=['валюта', 'Обновление резюме'],
                         right_on=['currency', 'date',])
#заполняем появившиеся пропуски в столбцах close и proportion единицей (1) 1 с помощью метода fillna().
hh_data = hh_data.fillna(value={'close': 1, 'proportion': 1})
#преобразуем тип данных в столбце желаемая ЗП'
hh_data['желаемая ЗП'] = hh_data['желаемая ЗП'].astype('float64')
#создаем столбец 'ЗП (руб)'
hh_data['ЗП (руб)'] = hh_data['желаемая ЗП']*hh_data['close']/hh_data['proportion']
#удаляем исходный столбец заработной платы «ЗП» и промежуточные столбцы
hh_data = hh_data.drop(['ЗП','currency', 'date', 'валюта'], axis=1)

In [ ]:
exrate_data = pd.read_csv('data/ExchangeRates.csv', sep = ',') #чтение файла с курсами валют
#print(exrate_data.head())#функция для выделения первого аргумента из строки
def salary_func(arg):
    salary = float(arg.split(' ')[0])
    return salary
#hh_data['валюта'] = hh_data['ЗП'].apply(lambda x: x.split(' ')[1].replace('.', ''))
#print(hh_data['валюта'].unique())
def currency_func(arg):
    #словарь возможных значений валют
    currency_dict = {'руб': 'RUB', 'белруб': 'BYN', 'KZT': 'KZT', 'EUR': 'EUR', 'USD': 'USD', 'грн': 'UAH', 'сум': 'UZS', 'KGS': 'KGS', 'AZN': 'AZN'}
    currency = arg.split(' ')[1].replace('.', '') #выделение первого аргумента из строки, удаление точки из названия валюты
    return currency_dict[currency] #возвращает значение словаря по ключу(аргументу)
hh_data['желаемая ЗП'] = hh_data['ЗП'].apply(salary_func) #создание столбца с желаемой суммой ЗП
hh_data['валюта'] = hh_data['ЗП'].apply(currency_func) #создание столбца с валютой желаемой ЗП
exrate_data['date'] = pd.to_datetime(exrate_data['date'], dayfirst=True).dt.date #изменение формата столбца с датой (только дата)
hh_data['Обновление резюме'] = pd.to_datetime(hh_data['Обновление резюме'], dayfirst=True).dt.date #изменение формата столбца с датой (только дата)#объединение двух баз данных (подстановка значений из таблицы с курсами валют по дате и валюте в основную таблицу)
hh_df = hh_data.merge(exrate_data,
                       left_on=['валюта', 'Обновление резюме'], #признаки, по которым ищем данные в основной таблице
                       right_on=['currency', 'date'], #признаки, по которым подставляем данные из таблицы с курсами валют
                       how='left') #все значения из основной таблицы#заполнение пустых значений "1" (для рублей)
hh_df['close'] = hh_df['close'].fillna(1)
hh_df['proportion'] = hh_df['proportion'].fillna(1)
#print(rate_date_cur.head())#добавлени столбца с желаемой ЗП на основании расчетов по данным из вспомогательной таблицы
#hh_data['желаемая ЗП'] = hh_data['желаемая ЗП'].astype('float64')
hh_df['ЗП (руб)'] = hh_df['close'] * hh_df['желаемая ЗП']/hh_df['proportion']

In [ ]:
def get_experience(arg):
    if arg is np.nan or arg == 'Не указано':
        return None
    year_words=['год', 'года', 'лет']
    month_words=['месяц', 'месяца', 'месяцев']
    arg_splitted = arg.split(' ')[:7]
    years = 0
    months = 0
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            years = int(arg_splitted[index-1])
        if item in month_words:
            months = int(arg_splitted[index-1])
    return int(years*12 + months)


hh_df['Опыт работы (месяц)'] = hh_data['Опыт работы'].apply(get_experience)
hh_df = hh_df.drop('Опыт работы', axis=1)

In [ ]:
#создаём словарь со столбцами, в которых надо удалить строки с пропусками 
values = {
    'Последняя/нынешняя должность':hh_df['Последняя/нынешняя должность'].median(),
    'BloodPressure':combine_data['BloodPressure'].median(),
    'Glucose':combine_data['Glucose'].median(),
    'BMI':combine_data['BMI'].median()
}
#заполняем оставшиеся записи константами в соответствии со словарем values
hh_df = hh_df.fillna(values)
#выводим результирующую долю пропусков
round(diabetes['SkinThickness'].mean(), 1)

In [ ]:

hh_df= hh_df['Последняя/нынешняя должность', 'Последнее/нынешнее место работы'].dropna(how='any', thresh=thresh, axis=0)
print(hh_df.shape)

In [ ]:

#hh_year = hh_df.copy()
#hh_year['Опыт работы (год)'] = hh_year['Опыт работы (месяц)']/12

#print(hh_year['Опыт работы (год)'][hh_year['Опыт работы (год)'] >= hh_year['Возраст']].count())

In [ ]:
#hh_df['Опыт работы (год)'] = hh_df['Опыт работы (месяц)']/12
#print(hh_df['Опыт работы (год)'][hh_df['Опыт работы (год)'] >= hh_df['Возраст']].count())

In [ ]:

#cleaned['Опыт работы (месяц)'] = cleaned.loc[:, 'Price']
#cleaned['Опыт работы (год)'] = cleaned['Опыт работы (месяц)']/12

#убираем из таблицы строки где Опыт работы превышает Возраст
#data_cleaned = cleaned[cleaned.loc[:, 'Опыт работы (год)'] < cleaned.loc[:, 'Возраст']]
#data_cleaned = data_cleaned.drop('Опыт работы (год)', axis=1) #удаляем созданный признак
#cleaned.shape[0] - data_cleaned.shape[0]


In [ ]:
#в логарифмическом масштабе
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
log_age= np.log(hh_df['Возраст'] + 1)

histplot = sns.histplot(data = log_age, bins=30, ax=axes[0])
histplot.set_title('Распределение возраста');

boxplot = sns.boxplot(data=log_age, ax=axes[1], orient='h');
boxplot.set_title('Распределение возраста');

histplot.axvline(log_age.mean(), color='k', lw=2)
histplot.axvline(log_age.mean()+ 3 * log_age.std(), color='k', ls='--', lw=2)
histplot.axvline(log_age.mean()- 3 * log_age.std(), color='k', ls='--', lw=2);

boxplot.axvline(log_age.mean(), color='k', lw=2)
boxplot.axvline(log_age.mean()+ 3 * log_age.std(), color='k', ls='--', lw=2)
boxplot.axvline(log_age.mean()- 3 * log_age.std(), color='k', ls='--', lw=2);

In [ ]:
def outliers_z_score_mod(data, feature, log_scale=True, left=3, right=4):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(hh_df, 'Возраст', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
hh_data = hh_data.drop('Город, переезд, командировки', axis=1)